In [1]:
#import support
import pandas as fu
import os
import glob
from datetime import datetime as dt
import json

Data for Subhub/MISP Keycloak data

type: str,
clientId: str,
userId: str,
ipAddress: str,
identity_provider_identity: str,
username: str,
loginDate: datetime

userId: str,
username: str,
org: str,
countSubhub: [ (day, #login), (day, #login)]




In [2]:
# sets the folder path to collect log data.
path1 = os.getcwd()
path2 = path1 + "\\Data\\"

#sets the raw imports.
data_list = glob.glob(os.path.join(path2, "*keycloak*.json"))

In [3]:
print('List of imported files:')
count = 0
for data in data_list:
    print(count, ': ', data.split('\\')[-1])
    count+=1

List of imported files:
0 :  (block) 20250118-20250512_keycloaklogs_114days.json
1 :  (block) 20250118-20250519_keycloaklogs_121days.json
2 :  20241213-20250313_keycloaklogs_30days_OPTION2.json
3 :  2024_12_18-2025-01_17_60days-90days_keycloaklogs.json
4 :  20250321-20250328_keycloaklogs_7days.json
5 :  20250328-20250404_keycloaklogs_7days.json
6 :  20250404-20250411_keycloaklogs_7days.json
7 :  20250411-20250418_keycloaklogs_full-log-groups_7days.json
8 :  20250418-20250425_keycloaklogs_7days.json
9 :  20250425-20250502_keycloaklogs_7days.json
10 :  2025_01_17-2025-02_16_30days-60days_keycloaklogs.json
11 :  2025_02_16-2025-03_18_0days-30days_keycloaklogs.json


In [9]:
# open files

content = []
for data in data_list:
    try:
        with open(data) as f:
            content.extend(json.load(f))
    except:
        #Prints which file failed to open, as well as gives you a location to go test why the json load possibly failed.
        print ("Failed: " + data)
        print ("Go to https://jsonlint.com/ to fix the error")

#Displays the total number of objects to be processed.
print("Total entries: " + str(len(content)))

Total entries: 111205


In [17]:
for entry in content:
    try:
        if '@message' in entry:
            pass
        else:
            print(entry)
    except:
        print('failed')


{'_section': 'MISP Logins'}
{'_section': 'OpenSearch Logins'}
{'_section': 'MISP Logins'}
{'_section': 'OpenSearch Logins'}


In [57]:
    
log_db = []
count = 0
tempDate = str
tempTime = str
logKeys = ['type', 'clientId', 'userId', 'username', 'identity_provider_identity']

idstore = []


for entry in content:

    tempLog = {}

    try:

        if '@message' in entry:

            tempLog['id'] = entry['@message']['json']['time']

            logitem = entry['@message']['json']['log'].split(',')

            tempDate, tempTime = entry['@message']['json']['time'].split('T')

            tempLog['year'] = int(tempDate.split('-')[0]) 
            tempLog['month'] = int(tempDate.split('-')[1])
            tempLog['day'] = int(tempDate.split('-')[2])        
            tempLog['time'] = tempTime.split('Z')[0]

            for item in logitem:
                if '=' not in item:
                    pass
                else:
                    item = item.split('=')
                    item[0] = item[0].split(' ')[-1]
                    if item[0] in logKeys:
                        tempLog[item[0]]= item[1]
            if 'type' in tempLog.keys():
                if tempLog['id'] not in idstore:
                    idstore.append(tempLog['id'])
                    log_db.append(tempLog)
    except:
        print(str(count) + '. Failed entry: ' + entry['@message'])
    count += 1


8284. Failed entry: changed: [i-0fe826bdf370cd47f] => {"changed": true, "cmd": ["sudo", "Format=up-prod", "MOUNTED=true", "/var/lib/cloud/scripts/per-boot/90-rk-start.sh", "/raid"], "delta": "0:04:55.857506", "end": "2025-02-03 23:53:21.735509", "msg": "", "rc": 0, "start": "2025-02-03 23:48:25.878003", "stderr": "WARNING! Your password will be stored unencrypted in /home/rk/.docker/config.json.\nConfigure a credential helper to remove this warning. See\nhttps://docs.docker.com/engine/reference/commandline/login/#credential-stores\n\ntime=\"2025-02-03T23:48:27Z\" level=warning msg=\"/RK/rk-prototype/docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion\"\ntime=\"2025-02-03T23:48:27Z\" level=warning msg=\"/RK/rk-prototype/docker-compose.gov.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion\"\ntime=\"2025-02-03T23:48:27Z\" level=warning msg=\"/RK/rk-prototype/doc

In [58]:
for item in log_db:
    print(item)
    break

{'id': '2025-01-21T15:13:46.66334097Z', 'year': 2025, 'month': 1, 'day': 21, 'time': '15:13:46.66334097', 'type': 'LOGIN', 'clientId': 'subhub', 'userId': '71009c21-2345-4867-8916-0aee303ed379', 'identity_provider_identity': 'robert.walter', 'username': 'robert.walter.civ'}


In [59]:
for log in log_db:
    for key in log:
        if key == 'clientId':
            if 'https' in log[key]:
                log[key] = 'metadata'
                #print(log[key])

In [60]:
userdataloc = glob.glob(os.path.join(path2, "response*.json"))
userList = []


try:
    with open(userdataloc[0], "r") as f:
        userList = json.load(f)
        #for line in f:
         #   line = line.strip('\n')
          #  if count == 0:
           #     userKey = line.split(',')
            #    count += 1
            #else:
             #   userList.append(dict(zip(userKey, line.split(','))))
                #print(userList[count-1])
              #  count+=1
            #userList.append(line)
            #print(line)
except:
    print('failed')

print(userList)

{'message': 'Retrieved users in system', 'data': {'Users': [{'email': 'ajspak@nsa.gov', 'username': 'aaron.spak.mil', 'firstName': 'Aaron', 'lastName': 'Spak', 'enabled': True, 'attributes': {'organization': 'rk', 'gov-organization': 'CNMF-JTF1', 'unclass-phone': '', 'misp-id': '105', 'status': 'New User', 'Role': 'Analyst'}, 'groups': ['rk'], 'realmRoles': ['default-roles-rk', 'Analyst'], 'federatedIdentities': None, 'id': '2e7e4907-6b29-4405-8586-37329eb7b00d'}, {'email': 'adam.b.duby.mil@army.mil', 'username': 'adam.duby.mil', 'firstName': 'Adam B', 'lastName': 'Duby', 'enabled': True, 'attributes': {'organization': 'rk', 'gov-organization': 'ARCYBER-CPB', 'unclass-phone': '', 'misp-id': '144', 'status': 'New User', 'Role': 'Analyst'}, 'groups': ['rk'], 'realmRoles': ['default-roles-rk', 'Analyst'], 'federatedIdentities': None, 'id': '70af281f-1249-486d-ac41-7e26ec85bd09'}, {'email': 'adtallm@cybercom.mil', 'username': 'adam.tallman.civ', 'firstName': 'Adam', 'lastName': 'Tallman', 

In [61]:
userdataloc = glob.glob(os.path.join(path2, "*.csv"))

print(userdataloc)

['c:\\Users\\jtoy0\\OneDrive\\DevOps\\RK user data\\Data\\User_Orgs.csv']


In [62]:
userOrgList = []

try:
    count = 0
    with open(userdataloc[0], 'r') as f:
        for line in f:
            line = line.strip('\n')
            line = line.rstrip('"')
            line = line.lstrip('"')
            #print(line)
            if count == 0:
                line = line.split('","')
                userKey = line
                #print(userKey)
                count += 1
            else:
                line = line.split('","')
                userOrgList.append(dict(zip(userKey, line)))
                #print(userOrgList[count-1])
                count+=1
        
except:
    pass

print(userOrgList)

[{'RK-Organization': 'MARFORCYBER', 'Summary': 'Rhodes, Jeremy Mark', 'Email': 'jeremy.rhodes.ctr@usmc.mil', 'Labels': 'RK-Analysis', 'Affiliation': 'Ctr', 'RK-Role': 'Analyst - Basic user, non-specialized role'}, {'RK-Organization': 'AFCYBER', 'Summary': 'Wolf, Lucas R.', 'Email': 'lucas.wolf@us.af.mil', 'Labels': 'RK-Analysis', 'Affiliation': 'Mil', 'RK-Role': 'Supervisor/Team lead - Assign and track analysis requests. Respond to Request for Information (RFI).'}, {'RK-Organization': 'MARFORCYBER', 'Summary': 'MCALLISTER, CHRIS, E', 'Email': 'CEMCALL@NSA.GOV', 'Labels': 'RK-Analysis', 'Affiliation': 'Mil', 'RK-Role': 'Mission Lead - Organization Leader'}, {'RK-Organization': 'US Marine Corps', 'Summary': 'Shipman, John, D', 'Email': 'john.shipman.ctr@usmc.mil', 'Labels': 'RK-Analysis', 'Affiliation': 'Ctr', 'RK-Role': 'Forensics - Analyze memory images or file system for signs of malicious processes or files.'}, {'RK-Organization': 'US Marine Corps', 'Summary': 'Pavia, Isaac, M', 'Ema

In [63]:
#print(userList[0])
userList2 = []
for n in userList['data']['Users']:
    #print(n)
    userList2.append(n)

#print(userList2)    
for u in userList2:
    print(u['id'] + ', ' + u['email'])
    

2e7e4907-6b29-4405-8586-37329eb7b00d, ajspak@nsa.gov
70af281f-1249-486d-ac41-7e26ec85bd09, adam.b.duby.mil@army.mil
8c5c9ee9-6ad1-4c94-a74e-f85d1255dce1, adtallm@cybercom.mil
24f3d183-fc26-4757-bbc4-31d3c6e64cda, alex.jeffery.ctr@usmc.mil
4fa660a0-436e-4ddb-95c7-99690ba1d147, alexander.j.mueller10.civ@army.mil
cc76aa30-4337-43b0-89ef-3f02a34b2e04, aowilde@cybercom.mil
965767af-2d03-41c1-9084-fdc377131031, alicia.k.murphy2.ctr@us.navy.mil
e4720225-9184-458f-b0b6-94c5cd324315, ray_amanda@bah.com
1d0853f5-4568-496f-a875-43d4660d2b6c, kaushiva_anant@bah.com
f814aafe-6a72-4f05-bce2-abace26246aa, andrew.c.clarke.ctr@usmc.mil
21b00006-1e2f-44be-b1b2-07d4769f179a, arie.j.ahquin.mil@usmc.mil
d6f7011a-cabf-4d91-bc94-b4cfaf82079d, andrew.h.rowley.civ@army.mil
6008177a-56db-4b73-8258-43f919021bc3, austin.may@usmc.mil
77253e6a-b433-45a4-8e0e-385aa84a083c, price_bailey@bah.com
6a87efbf-8112-4a0d-ac1a-a3a4a90802a8, benjamin.lim1@usmc.mil
5783aeab-35b1-45b1-b307-4d9d2600d6b4, billy.gibson.ctr@usmc.mil

In [64]:
for user in userList2:
    print(user)

{'email': 'ajspak@nsa.gov', 'username': 'aaron.spak.mil', 'firstName': 'Aaron', 'lastName': 'Spak', 'enabled': True, 'attributes': {'organization': 'rk', 'gov-organization': 'CNMF-JTF1', 'unclass-phone': '', 'misp-id': '105', 'status': 'New User', 'Role': 'Analyst'}, 'groups': ['rk'], 'realmRoles': ['default-roles-rk', 'Analyst'], 'federatedIdentities': None, 'id': '2e7e4907-6b29-4405-8586-37329eb7b00d'}
{'email': 'adam.b.duby.mil@army.mil', 'username': 'adam.duby.mil', 'firstName': 'Adam B', 'lastName': 'Duby', 'enabled': True, 'attributes': {'organization': 'rk', 'gov-organization': 'ARCYBER-CPB', 'unclass-phone': '', 'misp-id': '144', 'status': 'New User', 'Role': 'Analyst'}, 'groups': ['rk'], 'realmRoles': ['default-roles-rk', 'Analyst'], 'federatedIdentities': None, 'id': '70af281f-1249-486d-ac41-7e26ec85bd09'}
{'email': 'adtallm@cybercom.mil', 'username': 'adam.tallman.civ', 'firstName': 'Adam', 'lastName': 'Tallman', 'enabled': True, 'attributes': {'organization': 'rk', 'gov-org

In [65]:
missingUser = ""
for user in userList2:
    orgFlag = False
    for org in userOrgList:
        if user['email'] == org['Email']:
            user['org'] = org['RK-Organization']
            txt = user['username']
            #print(f'Username: {txt}')
            orgFlag = True
    if orgFlag == False:
        #print(user['username'])
        txt = user['username']
        missingUser = missingUser + txt + ', '

missingUser = missingUser.rstrip(', ')
missingUser = missingUser.split(', ')

In [66]:

count = 1

for user in missingUser:
    print(str(count)+ ': ' + user)
    count+=1

#print(len(missingUser.split(',')))


1: amanda.ray.ctr
2: anant.kaushiva.ctr
3: bailey.price.ctr
4: burgis.ctr
5: christopher.loratto.ctr
6: davin.shearer.ctr
7: eric.noriega.ctr
8: ethan.hart.ctr
9: francesco.nicotra.ctr
10: ian.carnaghan.ctr
11: james.decoster.2.mil
12: jared.johnson.ctr
13: jooyong.lee.ctr
14: kateila.reese.ctr
15: kyle.aubuchon.ctr
16: lauren.fuller.ctr
17: mark.blomquist.ctr
18: micah.andres.ctr
19: michael.funk.ctr
20: myisha.mitchell.ctr
21: robert.miller.ctr
22: vincenzo.pierorazio.ctr
23: zabeltimothy.ctr
24: zachary.rom.ctr


In [67]:
missingOrg = ''
txt = ''
for org in userOrgList:
    userFlag = False
    for user in userList2:
        if org['Email'] == user['email']:
            userFlag = True
            #print 
            pass
    if userFlag == False:
        txt = org['Email']
        missingOrg = missingOrg + txt + ', '

missingOrg = missingOrg.rstrip(', ')
missingOrg = missingOrg.split(', ')



In [68]:
count = 1
for user in missingOrg:
    print(str(count) +': '+user)
    count+=1

#print(len(missingOrg.split(',')))

1: jeremy.rhodes.ctr@usmc.mil
2: lucas.wolf@us.af.mil
3: CEMCALL@NSA.GOV
4: john.shipman.ctr@usmc.mil
5: isaac.pavia.ctr@usmc.mil
6: payton.eoff.mil@usmc.mil
7: dmshear@cybercom.mil
8: jcdecos@cybercom.mil
9: micah.l.andres.ctr@army.mil


In [69]:
UserInfo = []
tempUser = {}

#UserInfo = [{'userId': 'f670759e-2af0-4114-977b-b3be9182e882', 'username': 'johnathon.murray.ctr'}]

for log in log_db:
    if 'userId' and 'username' in log.keys():
        tempUser = {'userId': log['userId'], 'username': log['username']}
    if tempUser in UserInfo:
        pass
    else:
        print(tempUser)
        UserInfo.append(tempUser)

try:
    UserInfo.remove({})
except:
    pass

print(len(UserInfo))

{'userId': '71009c21-2345-4867-8916-0aee303ed379', 'username': 'robert.walter.civ'}
{'userId': '0df60880-00c3-4a9f-b277-534268a21ff8', 'username': 'francesco.nicotra.ctr'}
{'userId': '1d0853f5-4568-496f-a875-43d4660d2b6c', 'username': 'anant.kaushiva.ctr'}
{'userId': 'd34e4f01-cee9-49dc-8610-03af6d4396ad', 'username': 'steven.brewer.mil'}
{'userId': '6646ab35-b8b7-470c-bd0c-a16e6a331bc1', 'username': 'willie.turner.ctr'}
{'userId': '649fcf96-e6dc-4ec0-9493-3c4836ea5078', 'username': 'eylia.gaitan.ctr'}
{'userId': 'f670759e-2af0-4114-977b-b3be9182e882', 'username': 'johnathon.murray.ctr'}
{'userId': '6e0b2440-9189-4165-9a74-4200cf94d84b', 'username': 'jrvand.civ'}
{'userId': '68d05df1-f1ca-440f-bc9d-a413fde8e7ef', 'username': 'kenneth.zellmer.civ'}
{'userId': '60b6af21-3f11-41f0-93c4-1cfd3bd5f62a', 'username': 'robert.chunn.mil'}
{'userId': '9451adf9-e844-421b-9f1a-25bf1334a44f', 'username': 'patrick.jones.ctr'}
{'userId': 'cad353c6-6b4d-4b5a-b3d6-674a77153946', 'username': 'dontaye.dav

In [70]:

for user in UserInfo:
    print(user['username'])

print(len(UserInfo))

robert.walter.civ
francesco.nicotra.ctr
anant.kaushiva.ctr
steven.brewer.mil
willie.turner.ctr
eylia.gaitan.ctr
johnathon.murray.ctr
jrvand.civ
kenneth.zellmer.civ
robert.chunn.mil
patrick.jones.ctr
dontaye.davis.mil
maverick.franco.ctr
james.agbai.ctr
gregory.yingling.1.civ
michael.funk.ctr
jared.johnson.ctr
winston.barber.civ
eric.olson.mil
hunter.haley.ctr
kyle.aubuchon.ctr
burgis.ctr
mark.blomquist.ctr
alicia.murphy.ctr
patrick.oloughlin.ctr
lauren.fuller.ctr
vincent.motley.mil
ian.carnaghan.ctr
sean.k.nagamine.ctr
gabriel.huber.mil
juandella.williams-gray.mil
kristopher.howard.ctr
amanda.ray.ctr
michael.laycock.mil
bryan.clack.ctr
kateila.reese.ctr
jordan.keith.mil
jaeron.jenkins
william.e.mcconville2.ctr
charles.clement.mil
samantha.miller.ctr
joseph.cobian.ctr
george.bell.civ
robert.miller.ctr
steven.hilton.ctr
paul.c.cagle3.civ
thomas.lieske.mil
darren.espree.mil
quinton.sloan.civ
rahmiere.jones.mil
ronald.davis.civ
zabeltimothy.ctr
jonathan.g.race.ctr
geoffrey.taylor.ctr
raymo

In [71]:
count = 0
for log in log_db:
    if 'username' not in log.keys():
        for user in UserInfo:
            if user['userId'] == log['userId']:
                log_db[count]['username'] = user['username']
                
    count+=1

In [73]:
count = 0
for log in log_db:
    for user in userList2:
        if user["id"] == log['userId']:
            #print(user['Gov Organization'])
            log_db[count]['RK-Org'] = user['attributes']['gov-organization']
            if user['username'] not in missingUser:
                #sets the org to what has been defined as the users owning org per the Request for acount ticket.
                log_db[count]['org'] = user['org']
            else:
                #sets org to vendor that is developing/maintaining the system.  Backend user accounts.
                log_db[count]['org'] = 'BAH' 
    count +=1


In [35]:
#I don't remember the purpose of this code, will need to research.

columnsList = []
count = 0

for item in log_db:
    print(item.keys())
    if 'type' in item.keys():
        print(item['type'])
    for k in item.keys():
        if k in columnsList:
            pass
        else:
            columnsList.append(k)

dict_keys(['id', 'year', 'month', 'day', 'time', 'type', 'clientId', 'userId', 'identity_provider_identity', 'username', 'RK-Org', 'org'])
LOGIN
dict_keys(['id', 'year', 'month', 'day', 'time', 'type', 'clientId', 'userId', 'identity_provider_identity', 'username', 'RK-Org', 'org'])
LOGIN
dict_keys(['id', 'year', 'month', 'day', 'time', 'type', 'clientId', 'userId', 'identity_provider_identity', 'username', 'RK-Org', 'org'])
LOGIN
dict_keys(['id', 'year', 'month', 'day', 'time', 'type', 'clientId', 'userId', 'identity_provider_identity', 'username', 'RK-Org', 'org'])
LOGIN
dict_keys(['id', 'year', 'month', 'day', 'time', 'type', 'clientId', 'userId', 'identity_provider_identity', 'username', 'RK-Org', 'org'])
LOGIN
dict_keys(['id', 'year', 'month', 'day', 'time', 'type', 'clientId', 'userId', 'identity_provider_identity', 'username'])
LOGIN
dict_keys(['id', 'year', 'month', 'day', 'time', 'type', 'clientId', 'userId', 'identity_provider_identity', 'username', 'RK-Org', 'org'])
LOGIN
di

In [74]:
df_loginfo = fu.DataFrame(log_db)

print(df_loginfo)

                                   id  year  month  day                time  \
0       2025-01-21T15:13:46.66334097Z  2025      1   21   15:13:46.66334097   
1      2025-01-21T15:49:59.490464711Z  2025      1   21  15:49:59.490464711   
2      2025-01-21T15:50:33.065665672Z  2025      1   21  15:50:33.065665672   
3      2025-01-21T16:06:18.992233515Z  2025      1   21  16:06:18.992233515   
4      2025-01-21T17:27:34.058537024Z  2025      1   21  17:27:34.058537024   
...                               ...   ...    ...  ...                 ...   
21303  2025-03-13T20:08:45.777038141Z  2025      3   13  20:08:45.777038141   
21304  2025-03-13T20:06:47.757706138Z  2025      3   13  20:06:47.757706138   
21305  2025-03-13T20:04:14.000672769Z  2025      3   13  20:04:14.000672769   
21306  2025-03-13T20:01:47.975548431Z  2025      3   13  20:01:47.975548431   
21307   2025-03-13T19:59:14.02129759Z  2025      3   13   19:59:14.02129759   

                                type clientId  \
0 

In [75]:

file_output = dt.now()

file_output = file_output.strftime('%Y%m%d') + '_UserInfo.xlsx'

print(file_output)


20250523_UserInfo.xlsx


In [76]:
df_loginfo.to_excel(file_output, sheet_name= 'logs', index=False)